In [ ]:
pip install git+https://github.com/mpi-astronomy/gdr3apcal

In [1]:
pip install astroquery

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   -----------------

In [1]:
from astroquery.simbad import Simbad
import numpy as np
import pandas as pd
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
from astropy import units as u
from astroquery.vizier import Vizier
import warnings

# Configure query settings
Simbad.TIMEOUT = 1200
Simbad.ROW_LIMIT = -1
Simbad.add_votable_fields('ids', 'otype', 'mesfe_h')  # Updated: 'fe_h' → 'mesfe_h'

Vizier.TIMEOUT = 1200
vizier = Vizier(columns=['TIC', 'GAIA', '_RAJ2000', '_DEJ2000'])
vizier.ROW_LIMIT = -1 

Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"
Gaia.ROW_LIMIT = -1

# Suppress warnings from Simbad (optional, but recommended for clean output)
warnings.filterwarnings('ignore', category=DeprecationWarning, module='astroquery.simbad.core')

def get_gaia_and_metallicity_from_simbad(target_ids):
    """Query SIMBAD for Gaia DR3 IDs and metallicity (Fe/H) with robust error handling."""
    s = Simbad.query_objects(target_ids)
    gaia_ids = np.full(len(target_ids), np.nan, dtype=object)
    fe_h_values = np.full(len(target_ids), np.nan, dtype=float)
    
    if s is None:
        print('No Simbad match found for any target_ids')
        return gaia_ids, fe_h_values
    
    print("Available columns in SIMBAD results:", s.colnames)
    
    for i, entry in enumerate(s):
        try:
            # Safely get IDS if available
            ids = entry['ids'].split('|') if 'ids' in entry.colnames else []
            
            # Extract Gaia DR3 ID
            for id_str in ids:
                if 'Gaia DR3' in id_str:
                    gaia_ids[i] = id_str.strip()
                    break  # Take the first Gaia DR3 ID found
            
            # Extract metallicity if available
            if 'mesfe_h.fe_h' in entry.colnames and not np.isnan(entry['mesfe_h.fe_h']):
                fe_h_values[i] = entry['mesfe_h.fe_h']
                
        except Exception as e:
            print(f"Error processing entry {i}: {str(e)}")
            continue
    
    return gaia_ids, fe_h_values


# Load TOI data
cols1 = ['toi', 'tid', 'pl_orbper', 'pl_trandurh', 'pl_trandep', 'pl_rade', 'pl_eqt', 'pl_insol',
         'st_teff', 'st_logg', 'st_rad', 'ra', 'dec', 'tfopwg_disp']

data = pd.read_csv('TOI_2025.08.02_05.03.17.csv', comment='#', usecols=cols1)

# Count items before/after cleaning
original_count = len(data)
data = data.dropna()
cleaned_count = len(data)
print(f"Original CSV entries: {original_count}")
print(f"Entries after dropna(): {cleaned_count}")

# Process TOI data
raw_toi_df = data.reset_index(drop=True)
raw_toi_df['tid'] = 'TIC ' + raw_toi_df['tid'].astype(str)

# Query SIMBAD for Gaia DR3 IDs and metallicity
gaia_ids, fe_h_values = get_gaia_and_metallicity_from_simbad(raw_toi_df['tid'])
raw_toi_df['Gaia_DR3_ID'] = gaia_ids
raw_toi_df['Fe_H'] = fe_h_values  # Column name remains 'Fe_H' for clarity

# Report missing Gaia IDs
missing_gaia = raw_toi_df['Gaia_DR3_ID'].isnull().sum()
print(f"{missing_gaia} out of {len(raw_toi_df)} TOIs lack Gaia DR3 IDs in SIMBAD")

# Report TOIs with Fe_H data
fe_h_found = (~raw_toi_df['Fe_H'].isnull()).sum()
print(f"{fe_h_found} out of {len(raw_toi_df)} TOIs have metallicity (Fe_H) data in SIMBAD")

# Save to new CSV
output_filename = "TOI_2025.08.02_FeH_Gaia3ID.csv"
raw_toi_df.to_csv(output_filename, index=False)
print(f"Results saved to {output_filename}")

Original CSV entries: 7658
Entries after dropna(): 6586
Available columns in SIMBAD results: ['main_id', 'ra', 'dec', 'coo_err_maj', 'coo_err_min', 'coo_err_angle', 'coo_wavelength', 'coo_bibcode', 'otype', 'ids', 'mesfe_h.bibcode', 'mesfe_h.catno', 'mesfe_h.compstar', 'mesfe_h.fe_h', 'mesfe_h.fe_h_prec', 'mesfe_h.flag', 'mesfe_h.log_g', 'mesfe_h.log_g_prec', 'mesfe_h.mespos', 'mesfe_h.teff', 'user_specified_id', 'object_number_id']


C:\Users\huangm\AppData\Local\Temp\ipykernel_24312\4040821091.py:50: UserWarning: Warning: converting a masked element to nan.
  fe_h_values[i] = entry['mesfe_h.fe_h']


Error processing entry 6586: index 6586 is out of bounds for axis 0 with size 6586
Error processing entry 6587: index 6587 is out of bounds for axis 0 with size 6586
Error processing entry 6588: index 6588 is out of bounds for axis 0 with size 6586
Error processing entry 6589: index 6589 is out of bounds for axis 0 with size 6586
Error processing entry 6590: index 6590 is out of bounds for axis 0 with size 6586
Error processing entry 6591: index 6591 is out of bounds for axis 0 with size 6586
Error processing entry 6592: index 6592 is out of bounds for axis 0 with size 6586
Error processing entry 6593: index 6593 is out of bounds for axis 0 with size 6586
Error processing entry 6594: index 6594 is out of bounds for axis 0 with size 6586
Error processing entry 6595: index 6595 is out of bounds for axis 0 with size 6586
Error processing entry 6596: index 6596 is out of bounds for axis 0 with size 6586
Error processing entry 6597: index 6597 is out of bounds for axis 0 with size 6586
Erro

In [2]:
import pandas as pd
from astroquery.vizier import Vizier
import numpy as np
from tqdm import tqdm

# Configure Vizier
Vizier.ROW_LIMIT = -1  # No row limit
Vizier.TIMEOUT = 60    # Timeout in seconds

# Load TOI data
cols1 = ['toi', 'tid', 'pl_orbper', 'pl_trandurh', 'pl_trandep', 'pl_rade', 'pl_eqt', 'pl_insol',
         'st_teff', 'st_logg', 'st_rad', 'ra', 'dec', 'tfopwg_disp']

df = pd.read_csv('TOI_2025.08.02_05.03.17.csv', comment='#', usecols=cols1)
tids = df['tid'].astype(str).tolist()

# Count items before/after cleaning
original_count = len(df)
df = df.dropna()
cleaned_count = len(df)
print(f"Original CSV entries: {original_count}")
print(f"Entries after dropna(): {cleaned_count}")

# Define catalogs with metallicity data (adjust as needed)
CATALOGS = {
    'APOGEE': 'IV/41',  # APOGEE DR17 (stellar abundances)
    'LAMOST': 'V/164',  # LAMOST DR7 (stellar parameters)
}

# Store results
fe_h_data = np.full(len(df), np.nan)  # Initialize with NaN

# Search for matches in each catalog
for catalog_name, catalog_id in CATALOGS.items():
    print(f"\nSearching in {catalog_name} ({catalog_id})...")
    
    try:
        # Query the entire catalog (may take time for large catalogs)
        catalogs = Vizier.get_catalogs(catalog_id)
        if not catalogs:
            print(f"No data found in {catalog_name}.")
            continue
        
        catalog = catalogs[0]  # Assuming the first table is the relevant one
        
        # Check if required columns exist
        if 'TIC' not in catalog.colnames or '[Fe/H]' not in catalog.colnames:
            print(f"Required columns (TIC or [Fe/H]) not found in {catalog_name}.")
            continue
        
        # Convert TIC column to string for matching
        catalog_tic = catalog['TIC'].astype(str).tolist()
        
        # Create a dictionary for fast lookup
        catalog_dict = {str(tic): feh for tic, feh in zip(catalog['TIC'], catalog['[Fe/H]'])}
        
        # Match TICs
        for idx, tid in enumerate(tqdm(tids, desc=f"Matching {catalog_name}")):
            if tid in catalog_dict:
                fe_h = catalog_dict[tid]
                if not np.isnan(fe_h):
                    fe_h_data[idx] = fe_h
                    
    except Exception as e:
        print(f"Error with {catalog_name}: {str(e)}")

# Add results to DataFrame
df['Vizier_Fe_H'] = fe_h_data

# Summary stats
total = len(df)
found = df['Vizier_Fe_H'].notna().sum()
missing = total - found

print("\n=== Results ===")
print(f"Total TIDs: {total}")
print(f"TIDs with [Fe/H] found: {found} ({found/total:.1%})")
print(f"TIDs with no [Fe/H]: {missing}")

# Save to CSV
output_file = "TOI_2025.08.02_Vizier_Fe_H.csv"
df.to_csv(output_file, index=False)
print(f"\nSaved to {output_file}")

Original CSV entries: 7658
Entries after dropna(): 6586

Searching in APOGEE (IV/41)...
Required columns (TIC or [Fe/H]) not found in APOGEE.

Searching in LAMOST (V/164)...
Error with LAMOST: ("Connection broken: ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)", ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

=== Results ===
Total TIDs: 6586
TIDs with [Fe/H] found: 0 (0.0%)
TIDs with no [Fe/H]: 6586

Saved to TOI_2025.08.02_Vizier_Fe_H.csv


In [5]:
# old codes, used in 2024

cols1 = ['toi', 'tid','pl_orbper', 'pl_trandurh', 'pl_trandep', 'pl_rade', 'pl_eqt', 'pl_insol',
        'st_teff', 'st_logg', 'st_rad', 'ra', 'dec', 'tfopwg_disp']

data = pd.read_csv('TOI_2025.08.02_05.03.17.csv', comment='#', usecols=cols1)
# Count the number of items in the original CSV file
original_count = len(data)

data = data.dropna()
# Count the number of items after dropping rows with missing values
cleaned_count = len(data)
print(f"Number of items in the original CSV file: {original_count}")
print(f"Number of items after executing data.dropna(): {cleaned_count}")

raw_toi_df = data.reset_index(drop=True)
raw_toi_df['tid'] = 'TIC ' + raw_toi_df['tid'].astype(str)

# Get Gaia ID from Simbad by crossmatching with TIC ID
gaia_ids_from_tic = get_gaia_source_id_with_target_id_from_simbad(raw_toi_df['tid'])
raw_toi_df['gaia_id'] = np.array(gaia_ids_from_tic).T

indices = raw_toi_df.index[raw_toi_df['gaia_id'].isnull()]
print(f'{len(indices)} out of {len(raw_toi_df)} TOIs do not have Gaia ID from TIC ID')

# Stop here, because if Gaia DR3 ID is not found from Simbad, it is not in Gaia DR3. Using cone search is not reliable.

Number of items in the original CSV file: 7358
Number of items after executing data.dropna(): 6330
1724 out of 6330 TOIs do not have Gaia ID from TIC ID


In [7]:
def get_parameters_from_gaia(gaia_id: np.array):
    parameter_query = ("""select sp.source_id, g.phot_g_mean_mag, g.mh_gspphot, g.teff_gspphot, g.logg_gspphot, g.azero_gspphot, g.ebpminrp_gspphot, g.ag_gspphot, 
                        sp.mg_gspphot, g.libname_gspphot, sp.alphafe_gspspec, sp.mh_gspspec, sp.fem_gspspec, sp.flags_gspspec, 
                        sp.radius_flame, sp.lum_flame, sp.mass_flame, sp.age_flame, r_med_geo, r_lo_geo, r_hi_geo
                        FROM gaiadr3.astrophysical_parameters AS sp
                        JOIN gaiadr3.gaia_source as g on sp.source_id = g.source_id
                        JOIN external.gaiaedr3_distance as dist on g.source_id = dist.source_id 
                        where 
                        sp.source_id in ("""+(len(gaia_id)-1)*"{},"+"{})").format(*gaia_id)
    job = Gaia.launch_job_async(parameter_query)
    r = job.get_results()

    return r.to_pandas()

In [9]:
df_with_gaia_id = raw_toi_df.dropna(subset=['gaia_id'])
df = get_parameters_from_gaia(df_with_gaia_id['gaia_id'].str.replace('Gaia DR3 ', '').astype(str))

INFO: Query finished. [astroquery.utils.tap.core]


In [11]:
df_with_gaia_id.loc[:, 'gaia_id'] = df_with_gaia_id.loc[:, 'gaia_id'].str.replace('Gaia DR3 ', '').astype(str)

# Convert SOURCE_ID to string before merging
df['SOURCE_ID'] = df['SOURCE_ID'].astype(str)

final_df = pd.merge(df_with_gaia_id, df, left_on='gaia_id', right_on='SOURCE_ID', how='left')
final_df.to_csv('TOI_2025.01.10_08.14.57_with_gaia_3.csv', index=False)

In [13]:
import gdr3apcal
import numpy
import pandas as pd

# some pandas data frame with your data from GACS using GACS column names
df = pd.read_csv('TOI_2025.01.10_08.14.57_with_gaia_3.csv')
# Instantiate calibration object
calib = gdr3apcal.GaiaDR3_GSPPhot_cal()
# Apply calibrations to [M/H] and/or Teff, returning a numpy array of calibrated values.
metal_calib = calib.calibrateMetallicity(df)

# Add the numpy array as a new column to the original DataFrame
df['calibrated_gspphot'] = metal_calib

# Save the updated DataFrame to a new CSV file
df.to_csv('TOI_2025.01.10_08.14.57_with_gaia_3.csv', index=False)

C:\Users\huangm\AppData\Local\anaconda3\Lib\site-packages\gdr3apcal\models\mars_mh.py: 682ec200c6392244c50e5659a9ff9923f6dd1f801e33fabb40a9fa93ba58c031
Model mh (C:\Users\huangm\AppData\Local\anaconda3\Lib\site-packages\gdr3apcal\models\mars_mh.py) input file does not match the configuration.Expecting bd102a2b38e161e6c770650e021d2d8696542e2bddd411d0a590fe28f24030fa, got 682ec200c6392244c50e5659a9ff9923f6dd1f801e33fabb40a9fa93ba58c031
Automatically adding cos(b) from given ra and dec assuming degrees.
Automatically adding cos(b) from given ra and dec assuming degrees.
Automatically adding cos(b) from given ra and dec assuming degrees.
Automatically adding cos(b) from given ra and dec assuming degrees.


In [15]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('TOI_2025.01.10_08.14.57_with_gaia_3.csv')

# Define the function to calculate calibrated gspspec
def calculate_calibrated_gspspec(row):
    if row['st_logg'] < 3.5:
        return row['mh_gspspec'] + 0.274 - 0.1373 * row['st_logg'] - 0.0050 * (row['st_logg'] ** 2) + 0.0048 * (row['st_logg'] ** 3)
    else:
        return row['mh_gspspec']

# Apply the function to each row in the DataFrame
df['calibrated gspspec'] = df.apply(calculate_calibrated_gspspec, axis=1)

# Save the updated DataFrame to the original CSV file
df.to_csv('TOI_2025.01.10_08.14.57_with_gaia_3.csv', index=False)

In [17]:
import numpy
import pandas as pd
import math

# Load the CSV file
df = pd.read_csv('TOI_2025.01.10_08.14.57_with_gaia_3.csv')

# Initialize counters
f_spec_count = 0
a_spec_count = 0
cal_phot_count = 0
nomet_count = 0
noconvert_count = 0

# Create new columns 'metallicity' and 'source_of_met'
df['metallicity'] = ''
df['source_of_met'] = ''

# Iterate through each row and apply the rules
for index, row in df.iterrows():
    if pd.notna(row['calibrated gspspec']) and pd.notna(row['fem_gspspec']):
        df.at[index, 'metallicity'] = row['mh_gspspec']+ row['fem_gspspec']
        df.at[index, 'source_of_met'] = 'f_spec'                                              
        f_spec_count +=1
    elif pd.notna(row['calibrated gspspec']) and pd.notna(row['alphafe_gspspec']):                                            
        df.at[index, 'metallicity'] = row['mh_gspspec']- math.log10(0.638 * 10**row['alphafe_gspspec']+0.362)
        df.at[index, 'source_of_met'] = 'a_spec'
        a_spec_count += 1
    elif pd.notna(row['calibrated_gspphot']):
        df.at[index, 'metallicity'] = row['calibrated_gspphot']
        df.at[index, 'source_of_met'] = 'cal_phot'
        cal_phot_count += 1
    elif pd.isna(row['mh_gspphot']) and pd.isna(row['mh_gspspec']):
        df.at[index, 'source_of_met'] = 'nomet'
        nomet_count += 1
    else:
        df.at[index, 'source_of_met'] = 'unconvert'
        noconvert_count += 1

# Print the counters
print(f"# of KOI with metallicity from spectroscopy converted based on [Fe/M]: {f_spec_count}")
print(f"# of KOI with metallicity from spectroscopy converted based on alphaFe: {a_spec_count}")
print(f"# of KOI with metallicity from calibrated photometry: {cal_phot_count}")
print(f"# of KOI with no metallicity as unable to convert [M/H] to [Fe/H]: {noconvert_count}")
print(f"# of KOI with no metallicity data: {nomet_count}")

# Save the updated DataFrame to a new CSV file
df.to_csv('TOI_2025.01.10_08.14.57_with_gaia_3.csv', index=False)

# of KOI with metallicity from spectroscopy converted based on [Fe/M]: 1339
# of KOI with metallicity from spectroscopy converted based on alphaFe: 1320
# of KOI with metallicity from calibrated photometry: 1328
# of KOI with no metallicity as unable to convert [M/H] to [Fe/H]: 272
# of KOI with no metallicity data: 347
